In [1]:
import pandas as pd
import sqlite3
import requests
from bs4 import BeautifulSoup
import csv
import time
from time import sleep
import os
import zipfile
# import re
# import glob

In [2]:
!rm -rf data

In [3]:
### Setup directories

# List of directories to be created
directories = [
#     './data/SR-game-logs',
#     './data/SR-opponent-game-logs',
#     './data/rosters',
    './data/play_by_play/'
]

# Loop through the directories and create each one if it doesn't already exist
for directory in directories:
    if not os.path.exists(directory):
        os.makedirs(directory)


In [4]:
# ### Copy ./data/nfl.db here from Scraping

!cp ../Scraping/nfl.db ./data
# !cp ../Scraping/data/games.csv ./data/Games.csv

In [5]:
# !python exportToCSV.py

import os
import pandas as pd
import sqlite3

# Check current working directory
print("Current working directory:", os.getcwd())

# Connect to SQLite Database
conn = sqlite3.connect('./data/nfl.db')  # Ensure this path is correct
cursor = conn.cursor()

# Attempt to save each table to a CSV file
try:
    for table_info in cursor.execute("SELECT name FROM sqlite_master WHERE type='table';"):
        table_name = table_info[0]
        df = pd.read_sql_query(f"SELECT * FROM {table_name};", conn)
        file_path = f'./data/{table_name}.csv'
        df.to_csv(file_path, index=False)
        print(f"Saved {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")

# Close the connection
conn.close()


Current working directory: /Users/td/Code/nfl-ai/Streamlit
Saved ./data/Teams.csv
Saved ./data/Games.csv
Saved ./data/PlayerStats.csv


In [6]:
### Make UpcomingGames.csv from ./data/nfl.db

import sqlite3
import pandas as pd

# Scraping file path
db_file_path = './data/nfl.db'

# Connect to the SQLite Scraping
conn = sqlite3.connect(db_file_path)

# Create a cursor object
cursor = conn.cursor()

# Query to select upcoming games for the 2023 season where both team scores are null
query_upcoming_games = """
SELECT * FROM Games 
WHERE season = 2023 AND away_score IS NULL AND home_score IS NULL;
"""

# Execute the query
cursor.execute(query_upcoming_games)
upcoming_games = cursor.fetchall()

# Column names for the Games table
cursor.execute("PRAGMA table_info(Games);")
columns = [column[1] for column in cursor.fetchall()]

# Convert the data to a DataFrame
df_upcoming_games = pd.DataFrame(upcoming_games, columns=columns)

# Save to a CSV file in the './data' directory
csv_file_path = './data/UpcomingGames.csv'
df_upcoming_games.to_csv(csv_file_path, index=False)

# Close the Scraping connection
conn.close()

# After running this code, the CSV file will be saved to the specified path.


In [7]:
import pandas as pd

# Load the NFL data from the CSV file
nfl_data = pd.read_csv('./data/Games.csv')

# Identify unplayed games in the 2023 season (games with missing scores)
# unplayed_games = nfl_data[(nfl_data['season'] == 2023) & (nfl_data['away_score'].isna() | nfl_data['home_score'].isna())]
unplayed_games = nfl_data[(nfl_data['season'] == 2023) & (nfl_data['away_score'].isna() & nfl_data['home_score'].isna())]

# Remove unplayed games from the dataset
nfl_data_cleaned = nfl_data.drop(unplayed_games.index)

# Save the cleaned dataset to a new CSV file in the same directory
nfl_data_cleaned.to_csv('./data/Games.csv', index=False)


In [8]:
### Download pbp 
### https://github.com/nflverse/nflverse-data/releases/tag/pbp

import requests

# Iterate through the years 2000 to 2023
for year in range(2000, 2024):
    # Construct the URL for the CSV file of the specific year
    url = f"https://github.com/nflverse/nflverse-data/releases/download/pbp/play_by_play_{year}.csv"
    
    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Open the file in write-binary mode and save the CSV
        with open(f"./data/play_by_play/play_by_play_{year}.csv", 'wb') as file:
            file.write(response.content)
        print(f"Downloaded and saved play_by_play_{year}.csv")
    else:
        print(f"Failed to download data for the year {year}")

# Zip the downloaded files
zip_filename = "./data/play_by_play.zip"
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for root, dirs, files in os.walk("./data/play_by_play/"):
        for file in files:
            zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), "./data/play_by_play/"))

print(f"Zipped all files into {zip_filename}")

Downloaded and saved play_by_play_2000.csv
Downloaded and saved play_by_play_2001.csv
Downloaded and saved play_by_play_2002.csv
Downloaded and saved play_by_play_2003.csv
Downloaded and saved play_by_play_2004.csv
Downloaded and saved play_by_play_2005.csv
Downloaded and saved play_by_play_2006.csv
Downloaded and saved play_by_play_2007.csv
Downloaded and saved play_by_play_2008.csv
Downloaded and saved play_by_play_2009.csv
Downloaded and saved play_by_play_2010.csv
Downloaded and saved play_by_play_2011.csv
Downloaded and saved play_by_play_2012.csv
Downloaded and saved play_by_play_2013.csv
Downloaded and saved play_by_play_2014.csv
Downloaded and saved play_by_play_2015.csv
Downloaded and saved play_by_play_2016.csv
Downloaded and saved play_by_play_2017.csv
Downloaded and saved play_by_play_2018.csv
Downloaded and saved play_by_play_2019.csv
Downloaded and saved play_by_play_2020.csv
Downloaded and saved play_by_play_2021.csv
Downloaded and saved play_by_play_2022.csv
Downloaded 